## RAG with AI Search and Foundry Agents

![lab_flow](./Assets/lab_flow.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting Up Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AzureAISearchAgentTool,
    PromptAgentDefinition,
    AzureAISearchToolResource,
    AISearchIndexResource,
    AzureAISearchQueryType,
)

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
ai_search_connection_name = os.getenv("AI_SEARCH_CONNECTION_NAME")
ai_search_index_name = os.getenv("AI_SEARCH_INDEX_NAME")

### Setting up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Fetching the AI Search Connection ID

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == ai_search_connection_name:
        connection_id = connection.id
        break

print(f"The AI Search Connection ID is: {connection_id}")

### Creating the RAG Agent 

In [ ]:
agent = project_client.agents.create_version(
    agent_name="RAG-Agent",
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions = "You are a helpful assistant that uses AI Search to answer user queries in a RAG setup",
        tools = [
            AzureAISearchAgentTool(
                azure_ai_search = AzureAISearchToolResource(
                    indexes = [
                        AISearchIndexResource(
                            project_connection_id = connection_id,
                            index_name = ai_search_index_name,
                            query_type = AzureAISearchQueryType.VECTOR_SEMANTIC_HYBRID,
                            top_k = 3
                        )
                    ]
                )
            )
        ]
    )
)

print(f"Created Agent: {agent.name} with version: {agent.version} and ID: {agent.id}")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Calling the Agent and Streaming the Response

In [ ]:
user_input = "what are the hotels offered by Margie's Travel in Las Vegas? State the Sources also please!"

# A Couple of more queries to try below:
# user_input = "Tell me about the strategic board composition of Margie's Travel."
# user_input = "Tell me about hotels offered by Margie's Travel in Dubai"

In [ ]:
response = openai_client.responses.create(
        tool_choice="required",
        conversation=conversation.id,
        input=user_input,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

print(f"Agent Response: {response.output_text}")
